### Pré-traitement des données

#### Classe et fonction pour charger les données

In [106]:
# Importer des bibliothèques
import argparse
import time
import csv
import csv
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

# Télécharger les ressources nécessaires
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()

# Fonction d'augmentation
def remplacer_par_synonymes(phrase):
    mots = nltk.word_tokenize(phrase)  # Tokenisation plus robuste avec NLTK
    nouvelle_phrase = []

    for mot in mots:
        # Lemmatisation du mot
        mot_lemme = lemmatizer.lemmatize(mot.lower())  # Lemmatisation et minuscule

        # Ignorer les mots trop courants ou les prépositions sans synonymes pertinents
        if mot_lemme in ['in', 'on', 'at', 'the', 'a', 'an', 'of', 'and', 'to']:
            nouvelle_phrase.append(mot)  # Ne pas remplacer ces mots
            continue

        synsets = wn.synsets(mot_lemme, lang="eng")  # Chercher les synonymes en anglais
        if synsets:
            # Prendre le premier synonyme si disponible
            synonyme = synsets[0].lemmas("eng")[0].name()
            nouvelle_phrase.append(synonyme)
        else:
            nouvelle_phrase.append(mot)  # Si pas de synonyme, garder le mot original

    return " ".join(nouvelle_phrase)


# Définir une classe pour stocker un seul exemple (instance) de sentiment (description, label)
class MemeExample:
    def __init__(self, label, description):
        self.description = description
        self.label = label

        #print(description)

    def __repr__(self):
        return repr(repr(self.label) + " : " + self.description)

    def __str__(self):
        return self.__repr__()


# Lit les exemples de sentiments au format [0 ou 1]<TAB>[phrase brute] ; tokenise et nettoie les phrases.
# Lire les exemples avec augmentation
def read_meme_examples(infile, augment_data=False, augmentation_factor=1):
    with open(infile, 'r') as file:
        reader = csv.reader(file, delimiter=",")
        exs = []
        for fields in reader:
            label = 1 if "1" in fields[0] else 0
            description = fields[1].lower()  # Convertir en minuscules

            # Ajouter des copies augmentées avec des synonymes
            for _ in range(augmentation_factor):
                if augment_data:  # Si l'augmentation est activée
                    exs.append(MemeExample(label, description))
                    description_augmente = remplacer_par_synonymes(description)  # Appliquer l'augmentation
                    exs.append(MemeExample(label, description_augmente))
                else:
                    exs.append(MemeExample(label, description))
    return exs[1:]  # Ignorer la première ligne d'en-tête

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\enzoc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\enzoc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\enzoc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### *Charger* les données

In [107]:
"""# Monter le lecteur pour accéder aux fichiers dans gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)"""

"# Monter le lecteur pour accéder aux fichiers dans gdrive\nfrom google.colab import drive\ndrive.mount('/content/gdrive', force_remount=True)"

In [108]:
#VOUS DEVEZ CHANGER LE CHEMIN DE train_file ET dev_file SELON OÙ VOUS LES STOCKEZ DANS VOTRE gdrive.


data_path = "../../preprocessing/"

#"TODO" changez le chemin pour train_file
train_file = data_path + "train.csv"
#"TODO" changez le chemin pour dev_file
dev_file = data_path + "test.csv"

# Charger les données des fichiers
train_exs = read_meme_examples(train_file, False)
dev_exs = read_meme_examples(dev_file, False)
n_pos = 0
n_neg = 0
for ex in train_exs:
    if ex.label == 1:
        n_pos += 1
    else:
        n_neg += 1
print("%d train examples: %d positive, %d negative" % (len(train_exs), n_pos, n_neg))

n_pos = 0
n_neg = 0
for ex in dev_exs:
    if ex.label == 1:
        n_pos += 1
    else:
        n_neg += 1
print("%d dev examples: %d positive, %d negative" % (len(dev_exs), n_pos, n_neg))

for ex in train_exs[:10]:
    print(ex)

for ex in dev_exs[:10]:
    print(ex)


10000 train examples: 5000 positive, 5000 negative
1000 dev examples: 500 positive, 500 negative
'0 : milk milkzip'
'1 : roses are red violets are blue if you dont say yes ill just rape you quickmemecom'
'0 : breaking news russia releases photo of donald trump with hooker in russian hotelwaitsorrywrong filenever mind'
'0 : man seeking woman ignad 18 o'
'0 : me explaining the deep lore of jrr tolkeins world of arda the prostitute i am paying to keep me company during covid quarantine 61'
'0 : pictophle app straight white malle starts talking puts headphones in  ears feminism 1 reply swipe up to view 5 replies chat agtinkta mah wwiem 501 rated chats 2 mi away rate this chat today 1429 the headphones invented by a while male dont ever fucking talk to me you mean on this smartphone app that was created by a white man or on your phone in general that was created by a white man im gonna call the police if you dont stop harassing me oh ok you mean get a white male to come help you gif say som

#### Indexer les exemples
Cette section contient le code d'un indexeur (Indexer) qui est utile pour créer un mappage entre les mots et les index. Il a déjà été implémenté pour vous. Lisez-le et essayez de comprendre ce que font ses méthodes.

In [109]:
# Bijection entre objets et entiers commençant à 0. Utile pour le mappage
# étiquettes, attributs(features), etc. en coordonnées d'un espace vectoriel.

# Cette classe crée un mapping entre des objets (ici des mots) et des index uniques
# Par exemple : apple->1, banana->2, etc.

class Indexer(object):
    def __init__(self):
        self.objs_to_ints = {}
        self.ints_to_objs = {}

    def __repr__(self):
        return str([str(self.get_object(i)) for i in range(0, len(self))])

    def __str__(self):
        return self.__repr__()

    def __len__(self):
        return len(self.objs_to_ints)

    # Renvoie l'objet correspondant à l'index particulier
    def get_object(self, index):
        if (index not in self.ints_to_objs):
            return None
        else:
            return self.ints_to_objs[index]

    def contains(self, object):
        return self.index_of(object) != -1

    # Renvoie -1 si l'objet n'est pas présent, l'index sinon
    def index_of(self, object):
        if (object not in self.objs_to_ints):
            return -1
        else:
            return self.objs_to_ints[object]

    # Ajoute l'objet à l'index s'il n'est pas présent, renvoie toujours un index non négatif
    def add_and_get_index(self, object, add=True):
        if not add:
            return self.index_of(object)
        if (object not in self.objs_to_ints):
            new_idx = len(self.objs_to_ints)
            self.objs_to_ints[object] = new_idx
            self.ints_to_objs[new_idx] = object
        return self.objs_to_ints[object]

### Definir le modèle de régression logistique

In [110]:
# Importer des bibliothèques
from collections import Counter
from typing import List
import numpy as np
import math

#### Definir l'extracteur d'attributs/features

In [111]:
# Type de base d'extraction d'attributs (features). Prend un exemple et renvoie une liste indexée d'attributs.
class FeatureExtractor(object):
    # Extraire les attributs (features). Inclut un indicateur add_to_indexer pour contrôler si l'indexeur doit être étendu.
    # Au moment du test, tous les attributs/features inconnus (non-vu auparavant) doivent être ignorés, mais au moment de l'entrainement,
    # nous voulons probablement continuer à l'étendre.
    def extract_features(self, ex, add_to_indexer):
        raise Exception("Don't call me, call my subclasses")


# Extrait les attributs (features) unigramme sac-de-mots (BOW) d'une phrase.
# Vous pouvez considérer un unigramme comme un mot unique (par exemple "love", "you").
# C'est à vous de décider comment vous voulez gérer les comptages
class UnigramFeatureExtractor(FeatureExtractor):
    def __init__(self, indexer: Indexer):
        self.indexer = indexer

    def extract_features(self, ex, add_to_indexer=False):
        features = Counter()
        for w in ex.description.split(): # Tokenisation par mot plutôt que par caractère (sans le split)
            feat_idx = self.indexer.add_and_get_index(w) \
                if add_to_indexer else self.indexer.index_of(w)
            if feat_idx != -1:
                features[feat_idx] += 1.0
        return features

#### Definir les classifieurs de base

In [112]:
# Type de base du classificateur de sentiment
class SentimentClassifier(object):
    # faire une prédiction pour l'exemple donné
    def predict(self, ex: MemeExample):
        raise Exception("Don't call me, call my subclasses")


# Prédit toujours la classe positive
class AlwaysPositiveClassifier(SentimentClassifier):
    def predict(self, ex: MemeExample):
        return 1

#### Classe régression logistique

In [113]:
class LogisticRegressionClassifier(SentimentClassifier):
    def __init__(self, feat_extractor: FeatureExtractor, train_examples, num_iters=50, reg_lambda=0.0, learning_rate=0.1):
        # TODO: Initialiser le modèle de régression logistique
        
        # Arguments : feat_extractor est unigram, train_examples est un jeu de données d'entrainement
        # num_iters est le nombre d'époques, reg_lambda est le paramètre de régularisation
        # learning_rate est le taux d'apprentissage utilisé dans la descente de gradient
        
        # ÉTAPE 1 : Définissez les variables pour les poids et les biais, et initialisez-les à zéro
        
        # ÉTAPE 2 : Appelez la fonction train(). (Cela a déjà été fait pour vous)

        ##### DÉBUT DE LA SOLUTION #####
        
        
        # Variable pour les poids
        nb_features = len(feat_extractor.indexer)
        self.weights = np.zeros(nb_features)
        
        # Variable pour le biais
        self.bias = 0
        
        # Permet d'utiliser le feature extractor dans la fonction predict 
        self.feat_extractor = feat_extractor
        
        ##### FIN DE LA SOLUTION  #####

        self.train(feat_extractor, train_examples, num_iters, reg_lambda, learning_rate)


    def train(self, feat_extractor: FeatureExtractor, train_examples, num_iters=50, reg_lambda=0.0, learning_rate=0.1, L2_regul=True):
        # TODO : fonction d'entraînement du modèle de régression logistique.
        # Utilisez une descente de gradient stochastique.

        ##### DÉBUT DE LA SOLUTION #####

        for epoch in range(num_iters):
            for example in train_examples:
                # Extraction des caracteristiques
                features = feat_extractor.extract_features(example, add_to_indexer=False)

                # Calcul de la sortie à l'aide du vecteur de poids, du biais ainsi que de la fonction d'activation (ici la fonction sigmoide)
                z = sum(self.weights[index] * count for index, count in features.items()) + self.bias
                p = 1 / (1 + np.exp(-z))

                # Calcul de l'erreur (différence entre la prédiction et la cible)
                error = example.label - p

                # Mise à jour des poids avec l'erreur obtenue
                for index, count in features.items():
                    if L2_regul:
                        self.weights[index] += error * count * learning_rate - (reg_lambda * self.weights[index])
                    else:
                        self.weights[index] += error * count * learning_rate


                # Mise à jour du biais
                self.bias += error * learning_rate


        ##### FIN DE LA SOLUTION  #####    

    def predict(self, example):
        # TODO : prédiction du modèle de régression logistique pour un seul exemple
        
        ##### DÉBUT DE LA SOLUTION #####              

        # Extraction des caracteristiques
        features = self.feat_extractor.extract_features(example, add_to_indexer=False)

        # Calcul de la sortie à l'aide du vecteur de poids, du biais ainsi que de la fonction d'activation (ici la fonction sigmoide)
        z = sum(self.weights[index] * count for index, count in features.items()) + self.bias
        p = 1 / (1 + np.exp(-z))
        
        classe = 1 if p >= 0.5 else 0
        
        return classe
        
        ##### FIN DE LA SOLUTION  #####

#### Fonction d'entrainement pour la régression logisique

In [114]:
# Entrainer un modèle de régression logsitique sur les exemples d'entrainement en utilisant le FeatureExtractor, tous donnés en paramètres.
def train_lr(train_exs: List[MemeExample], feat_extractor: FeatureExtractor, reg_lambda) -> LogisticRegressionClassifier:
    # TODO : fonction d'entraînement du modèle de régression logistique.
    # Remplissez le feature_extractor.
    # Initialisez et renvoiez un objet d'instance LogisticRegressionClassifier
    
    ##### DÉBUT DE LA SOLUTION #####
    
    # Remplissage du feature extractor
    for example in train_exs:
        feat_extractor.extract_features(example, add_to_indexer=True)
    
    # Instanciation du classifieur
    classifier = LogisticRegressionClassifier(feat_extractor, train_exs, num_iters=50, reg_lambda=reg_lambda, learning_rate=0.1)
    
    return classifier
    ##### FIN DE LA SOLUTION  #####

In [115]:
# POINT D'ENTREE PRINCIPAL pour vos modifications. 
# Entraîne et retourne un des modèles possibles selon les options passées.
def train_model(feature_type, model_type, train_exs, reg_lambda=0.1):
    
    # Initialiser le feature extractor
    if feature_type == "unigram":
        feat_extractor = UnigramFeatureExtractor(Indexer())
    else:
        raise Exception("Pass unigram")

    # entrainer le modèle
    if model_type == "AlwaysPositive":
        model = AlwaysPositiveClassifier()
    elif model_type == "LogisticRegression":
        model = train_lr(train_exs, feat_extractor, reg_lambda=reg_lambda)
    else:
        raise Exception("Pass AlwaysPositive or LogisticRegression")
    return model

### Fonctions pour l'évaluation du modèle

In [116]:
# Évalue un classificateur donné sur les exemples donnés
def evaluate(classifier, exs):
    return print_evaluation([ex.label for ex in exs], [classifier.predict(ex) for ex in exs])


# Imprime la précision en comparant la vérité du terrain (ground truth) - golds - et les prédictions, chacune étant une séquence d'étiquettes 0/1.
def print_evaluation(golds, predictions):
    num_correct = 0
    num_pos_correct = 0
    num_pred = 0
    num_gold = 0
    num_total = 0
    if len(golds) != len(predictions):
        raise Exception("Mismatched gold/pred lengths: %i / %i" %
                        (len(golds), len(predictions)))
    for idx in range(0, len(golds)):
        gold = golds[idx]
        prediction = predictions[idx]
        if prediction == gold:
            num_correct += 1
        if prediction == 1:
            num_pred += 1
        if gold == 1:
            num_gold += 1
        if prediction == 1 and gold == 1:
            num_pos_correct += 1
        num_total += 1

    print("Accuracy: %i / %i = %.2f %%" %
          (num_correct, num_total,
           num_correct * 100.0 / num_total))
    return num_correct * 100.0 / num_total
    
# ENTRÉE PRINCIPALE POUR L'ÉVALUATION sur les jeux de données d'entrainement et de développement
def eval_train_dev(model):
    print("===== Train Accuracy =====")
    train_acc = evaluate(model, train_exs)
    print("===== Dev Accuracy =====")
    eval_acc = evaluate(model, dev_exs)
    return [train_acc, eval_acc]

### Évaluation du modèle avec une représentation unigramme Bag-of-Words

In [117]:
# Entrainement du modèle
lr_unigram_model = train_model('unigram', 'LogisticRegression', train_exs)

In [119]:
"""def print_predictions(classifier, exs):
    for ex in exs:
        print(f"{ex.label} : {classifier.predict(ex)} : {ex.description}")

print_predictions(lr_unigram_model, dev_exs)"""

def compute_prediction_distribution(classifier, exs, dataset_name="Dataset"):
    predictions = [classifier.predict(ex) for ex in exs]
    num_zeros = sum(1 for p in predictions if p == 0)
    num_ones = sum(1 for p in predictions if p == 1)

    print(f"=== Distribution des prédictions sur {dataset_name} ===")
    print(f"Classe 0: {num_zeros} ({num_zeros / len(predictions) * 100:.2f}%)")
    print(f"Classe 1: {num_ones} ({num_ones / len(predictions) * 100:.2f}%)")
    print("========================================")

# Calcul de la distribution des prédictions
compute_prediction_distribution(lr_unigram_model, train_exs, "Train")
compute_prediction_distribution(lr_unigram_model, dev_exs, "Dev")
print()

# Evaluation des prédictions
accu = eval_train_dev(lr_unigram_model)

=== Distribution des prédictions sur Train ===
Classe 0: 7012 (70.12%)
Classe 1: 2988 (29.88%)
=== Distribution des prédictions sur Dev ===
Classe 0: 786 (78.60%)
Classe 1: 214 (21.40%)

===== Train Accuracy =====
Accuracy: 7924 / 10000 = 79.24 %
===== Dev Accuracy =====
Accuracy: 572 / 1000 = 57.20 %


[79.24, 57.2]